In [4]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model #Modelni yuklab olish uchun
import cvlib as cv #Face detect qiladigon library githubda bor
from tensorflow.keras.preprocessing.image import img_to_array #numpyni kerasni arrayiga otkazish uchun

In [ ]:
model = load_model("glass_Murodil.model") #modelniyuklash
turi = ['Glass', "without_glass"] #Klasiffikatsiya turlari. Yani dataset papka nomlari

In [3]:
pip install cvlib

  Created wheel for cvlib: filename=cvlib-0.2.7-py3-none-any.whl size=10046386 sha256=b0d6d3da36d7cf121a6b459fb51c614dbe9883457d81375bd121aa73d4f581f2
  Stored in directory: c:\users\cengineer\appdata\local\pip\cache\wheels\96\ad\4c\d80d4bf4143c154ab297e2384915c89aff43d77d1787b9a702
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=6cb46713104919c2ea983a9ad7e6bfe62312c3e1737993077eb0902b17022f25
  Stored in directory: c:\users\cengineer\appdata\local\pip\cache\wheels\d7\d9\89\a3f31c76ff6d51dc3b1575628f59afe59e4ceae3f2748cd7ad
Successfully built cvlib progressbar
Note: you may need to restart the kernel to use updated packages.


# OpenCv kodi


In [17]:
video = cv2.VideoCapture('test_video.mp4') 

while video.isOpened(): 
    
    boolen, kadr = video.read()
     
    #Kadrdan yuzni topib uni face ozgaruvchiga uzatadi. Yani 2 olchamli rasm matrixi uzatiladi    
    face,confidence = cv.detect_face(kadr)
    #print(face)
    
    
    #Kelayotgan face [a,b,c,d] matrix yani [starx,stary,endx,endy] ga index qoyib beradi
    for index, yuz in enumerate(face): 
        
        #Topilgan yuzni nuqtalarini alohida tenglab olamiz.
        (startX, startY, endX, endY) = yuz[0],yuz[1],yuz[2],yuz[3]
        
        #Kadrdagi nuqtalarni numpy matrixga (array)ga o'tkazib olamiz.
        yuz_np = np.copy(kadr[startX:endX, startY:endY]) #Xdan tugash x gacha Y dan tugash Y gacha
        
       
        
        #Face topilgan yuza kichkina bo'lsa amal bajarmaymiz. Ya'ni uzoqdagi yuzlarni topmasin.
        if yuz_np.shape[0] < 10 or yuz_np.shape[1] < 10:
            continue
        
        
        yuz_np = cv2.resize(yuz_np, (96,96)) #Yuz numpy ni imageni razmerini o'zgartiramiz 
        
        yuz_np = yuz_np.astype("float")/255.0 #Ranglar 255 gacha bo'lgani uchun matrixni foizda chiqarib beradi   
     
        yuz_np = img_to_array(yuz_np) #Tepada kutubxonani chaqirib, Rasm matrixni kerasni arrayiga otkazib olamiz
        
        #Yuz np ni dimensionini qoshvolamiz. Chunki predict metodiga kerakligi uchun
        yuz_np = np.expand_dims(yuz_np,axis=0) 
        
        #Eng birinchi matrix ni bashoratga yuboradi
        bashorat = model.predict(yuz_np)[0]
        
        #Bashoratdan kelayotgan 2 qiymatni qaysi biri katta bolsa oshani indexni topib beradi va indexga ozlashtiradi
        index = np.argmax(bashorat)
       
        #qaysi biri kelsa turidagi qiymat index lardan olib labelga tenglaydi.
        label = turi[index]
        
        #Agar label qiymati Glass papka nomiga teng bolsa 
        if label == "Glass":         
            color = (0,255,0) #yashil
        else: 
            color = (255,0,0) #Aks holda ko'k qaytarsin.
            
        #Labelni bashoratdan kelgan qiymatni foizga o'tkazib olinadi. Masalan: Glass:99.9 chiqadi
        label = f"{label}:{np.around(bashorat[index]*100,2)}"
        
        #Yozuvni tog'irlash  
        if startY-10 > 10: #StarY-10 nuqta 10 dan katta bo'lsa 
            Y = startY - 10 #Y nuqtani coordinatasini tushiramiz
        else: 
            Y = startY + 10 #Y nuqtani coordinatasini ko'taramiz. 
        
        #Topiglan Facega Tortburchak chizadi
        cv2.rectangle(kadr,(startX, startY),(endX,endY),color, 2)
        #Bashoratni yozuv qilib chiqaradi
        cv2.putText(kadr, label, (startX,Y),cv2.FONT_HERSHEY_COMPLEX, 1, color, 2)
        
        
    cv2.imshow("frame", kadr)
    
    # Q bosilguncha kutib turadi. 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video.release()        
cv2.destroyAllWindows()